# Train a new DLC network using labeled frames

This notebook provides instruction on how to train a DLC model after labeling frames. The pylids package works on top of the [DeepLabCut](https://github.com/DeepLabCut/DeepLabCut) framework. If you are not familiar with it please check this [link](https://deeplabcut.github.io/DeepLabCut/README.html). Also please run the `select_frames_to_label.ipynb` notebook to select the frames to label before running this notebook.

In [ ]:
import deeplabcut
import pylids
import os

In [ ]:
#path to DLC config file for your project
path_config_file = 'add/path/to/dlc/config.yaml'

### Create a training dataset for training the network

You can verify if this process went through correctly by checking the training-datasets folder inside your DLC project path

In [ ]:
#create a training dataset
deeplabcut.create_training_dataset(path_config_file)

### For training a network from ImageNet weights
Run the cell below if you want to train a new model. This step will edit the pose_config.yaml file DLC uses to modify the network training parameters. We will initialize the network with ImageNet model weights and set the appropriate learning rate. For our experiment we set the augmentations to False. You can set it to True if you want to use the augmentations.

In [ ]:
#Point to the DLC training protocol and explain config and pose config files

#read DLC pose config file
cfg=deeplabcut.auxiliaryfunctions.read_plainconfig(path_config_file)
trainposeconfigfile, testposeconfigfile, snapshotfolder=deeplabcut.return_train_network_path(path_config_file, shuffle=1)
cfg_dlc=deeplabcut.auxiliaryfunctions.read_plainconfig(trainposeconfigfile)

#pick that as large as your GPU can handle it (lower this if you run out of memory)
cfg_dlc['batch_size']=8

#set all augmentations to be false 
cfg_dlc['elastic_transform']=False
cfg_dlc['rotation']=0
cfg_dlc['covering']=False
cfg_dlc['motion_blur'] = False
cfg_dlc['mirror'] = False

#use adam optimizer instead of sgd
cfg_dlc['optimizer'] ="adam"

#Change the learning rate
cfg_dlc['multi_step']=[[0.0001, 40000], [5e-05, 60000], [1e-5, 120000]]

deeplabcut.auxiliaryfunctions.write_plainconfig(trainposeconfigfile,cfg_dlc)

### For fine-tuning a network
Run the cell below if you want to fine tune a model. This step will edit the pose_config.yaml file DLC uses to modify the network training parameters. We will initialize the network with the baseline model weights and fine tune it using a lower learning rate

In [ ]:
#read DLC pose config file
cfg=deeplabcut.auxiliaryfunctions.read_plainconfig(path_config_file)
trainposeconfigfile, testposeconfigfile, snapshotfolder=deeplabcut.return_train_network_path(path_config_file, shuffle=1)
cfg_dlc=deeplabcut.auxiliaryfunctions.read_plainconfig(trainposeconfigfile)

#pick that as large as your GPU can handle it (lower this if you run out of memory)
cfg_dlc['batch_size']=8

#initilize weights with the weights from the pretrained baseline model
cfg_dlc['init_weights'] = 'path/to/weights/dlc-models/iteration-0/project_name/train/snapshot-120000'

#set all augmentations to be false 
cfg_dlc['elastic_transform']=False
cfg_dlc['rotation']=0
cfg_dlc['covering']=False
cfg_dlc['motion_blur'] = False
cfg_dlc['mirror'] = False

#use adam optimizer instead of sgd
cfg_dlc['optimizer'] ="adam"

#Change the learning rate
cfg_dlc['multi_step']=[[3e-04, 3000]]

deeplabcut.auxiliaryfunctions.write_plainconfig(trainposeconfigfile,cfg_dlc)

### Train the network

Using a GPU, fine tuning a network takes less than an hour while training a network from ImageNet weights takes about half a day.

In [ ]:
#Trains the new DLC network
deeplabcut.train_network(path_config_file, shuffle=1, displayiters=100, saveiters=500, max_snapshots_to_keep=6)

### Analyze new eye videos using pylids based on the trained DLC network

In [ ]:
#Use pylids to analyze the videos

#path to the video to analyze
video2analyze = 'path/to/eye/video1'

#path to the folder where the results will be saved
save_folder = 'path/to/save/folder'

pylids_out = pylids.analyze_videos(eye_video = video2analyze,
                     model_name = path_config_file,
                    destfolder=os.path.join(save_folder))

You can see examples on how to use and analyze the output in the pylids_demo.ipynb notebook.